In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import codecs
import collections
import difflib
import numpy as np
from itertools import chain 
from os import listdir
from os.path import join, isfile
import pandas as pd
import re

In [11]:
in_path = 'Data/Nolink_txts/'
in_files = [f for f in listdir(in_path) if isfile(join(in_path, f))]
in_files = sorted(in_files)

txts = []
for file_name in in_files:
    article = open(join(in_path,file_name), "r", encoding='utf8').read()
    full = article
    txts.append(full)

In [12]:
# szotar létrehozása {id:szoveg}
id_txt = {file_name: open(join(in_path,file_name), "r", encoding = "UTF8").read() for file_name in in_files}

In [14]:
# szotar: {szoveg:{id set}}
rev_dict = {} 
for key, value in id_txt.items(): 
    rev_dict.setdefault(value, set()).add(key) 

In [ ]:
sim_dict = {}                   # szotar: {id: {hasonlo szovegek id set}}
already_seen = set()            # azok a szovegek, amikre már néztünk vagy találtunk hasonlóságot
# végigmegyünk a szövegegeken és megnézzük, hogy az aktuális szövegre hasonlít-e valamelyik a még nem látottak közül
for i, txt in enumerate(txts):  
    if txt not in already_seen:
        already_seen.add(txt)
        sims = difflib.get_close_matches(txt, txts[i+1:], cutoff = 0.7)  # a cutoff paraméterrel lehet állítani, hogy milyen erős hasonlóság a küszöb
        if len(sims) > 0:           # ha van hasonló, akkor hozzáadjuk a látottakhoz + felvesszük sim_dict-be
            print(i, rev_dict[txt], txt)
            for sim in sims:
                print(rev_dict[sim], sim)
                already_seen.add(sim)
                sim_dict.setdefault(list(rev_dict[txt])[0], set()).update(rev_dict[sim])
            print('-'*50)

2 {'10'} A József Attila Színház igazgatója, Nemcsák Károly volt az ATV vendége Rónai Egon Húzos című műsorában. A szombaton megjelenő műsorban szóba került a Színház- és Filmművészeti Egyetem ügye is, ebbe kapott betekintőt . a  Nemcsák a műsorban arról beszélt, hogy az SZFE-ügy „tudatosan előkészített dolog volt a balliberális oldalról”, illetve párhuzamot vont Vidnyánszky Attila és saját maga közt mondván, Vidnyánszkynak ugyanúgy nem szavaznak bizalmat, mint neki anno, amikor a József Attila élére került. A színész kiemelte, szerinte tudatosan előkészített kampány része volt az SZFE-ügy, -fejtette ki. Rónai visszakérdezésére, miszerint csak az egyik oldalról történik-e ez Nemcsák szerint, a színész azt mondta, „most ebben a pillanatban azt kell mondanom, hogy igen, a balliberális oldalról.” Gondolatmenetében kitért arra a kérdésére is, miszerint Nemcsák kifejtette azt is, hogy úgy saccolja, még öt évig lesz igazgató, aztán „át kell adni a stafétát a fiataloknak”. Beszélt emellett ar

In [ ]:
toremove = []     # az eltavolitando szovegek id-i a sim_dict alalpján
for kezd_id, hasn_idk in sim_dict.items():
    for hasn_id in hasn_idk:
        if hasn_id != kezd_id:      # ha teljes hasonlóság van, akkor a kezd_id is szerepel a setben, de az egyiket meg szeretnénk tartani
            toremove.append(i)
print(len(toremove))
print(toremove)

In [ ]:
for id in toremove:     # kiszedjük azokat a szövegeket, amik duplikátumok
    
    id_txt.pop(id, None)
endings = [t[-50:] for i, t in id_txt.items()]

In [ ]:
collections.Counter(endings)

In [ ]:
ending_d= {}        # szótár: {szövegvég:{id set}}
for i, file_name in enumerate(in_files):
    if file_name not in toremove:
        full = open(join(in_path,file_name), "r").read()
        full = full.replace('\n', ' ')
        ending_d.setdefault(full[-50:], set()).add(file_name)

In [ ]:
ending_temp = set()     # többször előforduló szövegvégek
for ending, file_names in ending_d.items():
    if len(file_names) > 1:
        print(f"\n{ending}-{len(file_names)}\n" \
            f"{df[df['id'].isin(list(file_names))][['Domain', 'Created']]}")
        ending_temp.add(ending)

In [ ]:
boilerplates = {}       # szótár {50 hosszú szövegvégek:teljes ismétlődő rész}
bps = []                #teljes ismétlődő szövegvégek
for ending in ending_temp:
    file_names = list(ending_d[ending])
    f1 = open(join(in_path,file_names[0]), "r").read()
    f2 = open(join(in_path,file_names[1]), "r").read()
    i = 49
    while f1[-i-1:] == f2[-i-1:]: #megkeres, hogy meddig egyezik
        i += 1
    ed = f1[-i+1:]
    # előző mondat befejezés eltávolítása
    if ed[0]=='.':
        ed = ed[1:]
    if ed[0]==' ':
        ed = ed[1:]
    if ed[0]=='\n':
        ed = ed[1:]
    boilerplates[ending] = ed
    bps.append(ed)
bps = tuple(bps)

In [ ]:
out_path = 'data/raw'
for i, file_name in enumerate(in_files):
    if file_name not in toremove:
    # beolvasás
        full = open(join(in_path,file_name), "r").read()
        full = full.replace('\n', ' ')

    #sablon végződések eltávolítása
    if full.endswith(bps):
        for ending in bps:
            if full.endswith(ending):
                full = full[:-len(ending)].strip()

    # ismétlődő részek keresése, és kivágása
    if full[:100] in full[100:]:
        full = full[re.search(re.escape(full[:100]), full[100:]).start()+100:].strip()
      
    # mentés
    with codecs.open(join(out_path, file_name), 'w', 'utf-8') as out_file:
        out_file.write(full)